# IMPORTING LIBRARIES

In [2]:
import pandas as pd
import requests
import time
import datetime
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import urllib

# IMPORTING LAST DATA

In [182]:
df_old = pd.read_csv('bwf_historic_rankings.csv').drop('Unnamed: 0', axis = 1)

/tmp/ipykernel_547/1555045639.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv('bwf_historic_rankings.csv').drop('Unnamed: 0', axis = 1)


In [183]:
df_old.head()

,country_code,country,continent,date,draw,rank,points,name
0,DEN,Denmark,Europe,2023-10-24,MS,1,107455,VIKTOR AXELSEN
1,INA,Indonesia,Asia,2023-10-24,MS,2,86611,ANTHONY SINISUKA GINTING
2,JPN,Japan,Asia,2023-10-24,MS,3,82015,KODAI NARAOKA
3,THA,Thailand,Asia,2023-10-24,MS,4,79778,KUNLAVUT VITIDSARN
4,CHN,China,Asia,2023-10-24,MS,5,78548,LI SHI FENG


In [184]:
df_old = df_old[df_old['country_code'].isna()]

In [185]:
df_old['date'].max()

'2023-10-16'

# SCRAPPING RANKING DATA

In [201]:
df_previous = pd.read_csv('export_temp.csv').drop('Unnamed: 0', axis = 1)
df_previous.head()

,0,1,2,3,4,5,6,7,8,9,10,date,discipline
0,1,NaN,NaN,DEN,[DEN] Viktor AXELSEN,NaN,25831,107455,15,Europe,Denmark,10/24/2023,MS
1,2,NaN,NaN,INA,[INA] Anthony Sinisuka GINTING,NaN,95661,86611,19,Asia,Indonesia,10/24/2023,MS
2,3,NaN,NaN,JPN,[JPN] Kodai NARAOKA,NaN,62063,82015,23,Asia,Japan,10/24/2023,MS
3,4,1.0,NaN,THA,[THA] Kunlavut VITIDSARN,NaN,64032,79778,18,Asia,Thailand,10/24/2023,MS
4,5,1.0,NaN,CHN,[CHN] LI Shi Feng,NaN,95476,78548,23,Asia,China,10/24/2023,MS


In [27]:
headers = {"User-Agent":"Mozilla/5.0"}

In [78]:
urls = ['https://bwf.tournamentsoftware.com/ranking/category.aspx?id=43340']

In [79]:
urls_ms = [x + '&category=472&C472FOC=&p=1&ps=100' for x in urls]
urls_ws = [x + '&category=473&C472FOC=&p=1&ps=100' for x in urls]
urls_md = [x + '&category=474&C472FOC=&p=1&ps=100' for x in urls]
urls_wd = [x + '&category=475&C472FOC=&p=1&ps=100' for x in urls]
urls_xd = [x + '&category=476&C472FOC=&p=1&ps=100' for x in urls]

In [85]:
df_scrap = pd.DataFrame(columns=df_previous.columns)
df_scrap.head()

,0,1,2,3,4,5,6,7,8,9,10,date,discipline


In [97]:
#i=0

for url in urls_xd:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    table_rows = soup.find_all('tr')

    l = []
    for row in table_rows:
            # Extract data from each row
        columns = row.find_all('td')
        for column in columns:
            l.append(column.text.strip())

    l = l[0:len(l)-1]

    n = 11

    temp_l = [l[i * n:(i + 1) * n] for i in range((len(l) + n - 1) // n )]

    df_temp = pd.DataFrame(temp_l)
    df_temp['date'] = soup.findAll('h3')[0].getText()[20:30]
    df_temp['discipline'] = 'XD'

    df_scrap = pd.concat([df_scrap, df_temp], ignore_index=True)

    #sleep(randint(2,10))
    #i+=1
    #print(i, '/', len(urls_ms))

In [202]:
df_previous.columns = ['rank', 'delta_rank', 'empty_1', 'country_code', 'name', 'empty', 'id', 'points', 'tournaments_played', 'continent', 'country', 'date', 'draw']
df_scrap.columns = ['rank', 'delta_rank', 'empty_1', 'country_code', 'name', 'empty', 'id', 'points', 'tournaments_played', 'continent', 'country', 'date', 'draw']

In [207]:
df_previous['date'] = df_previous['date'].apply(lambda x: x if (')' not in x or len(x.split('/')[0]) == 2) else '0'+x.replace(')', ''))
df_previous['date'] = df_previous['date'].apply(lambda x: (x[0:3] + '0' + x[3:10]) if len(x.split('/')[1].split('/')[0]) == 1 else x)
df_previous['date'] = df_previous['date'].str.replace(')', '')

In [208]:
df_previous['date'].unique()

array(['10/24/2023', '10/31/2023', '11/07/2023', '11/14/2023',
       '11/21/2023', '11/28/2023', '12/05/2023', '12/12/2023',
       '12/19/2023', '12/26/2023', '01/02/2024', '01/09/2024',
       '01/16/2024', '01/23/2024', '01/30/2024', '02/06/2024',
       '02/13/2024', '02/20/2024', '02/27/2024', '03/05/2024',
       '03/12/2024', '03/19/2024', '03/26/2024', '04/02/2024',
       '04/09/2024', '04/16/2024', '04/23/2024', '04/30/2024',
       '05/07/2024', '05/14/2024', '05/21/2024', '05/28/2024',
       '06/04/2024', '06/11/2024', '06/18/2024', '06/25/2024',
       '07/02/2024', '07/09/2024', '07/16/2024', '07/23/2024',
       '07/30/2024', '08/06/2024', '08/13/2024', '08/20/2024',
       '08/27/2024', '09/03/2024', '09/10/2024', '09/17/2024',
       '09/24/2024', '10/01/2024', '10/08/2024', '10/15/2024',
       '10/22/2024', '10/29/2024', '11/05/2024', '11/12/2024',
       '11/19/2024'], dtype=object)

In [209]:
df_delta = pd.concat([df_previous, df_scrap], ignore_index=True)
display(df_delta.head())
display(df_delta.tail())

,rank,delta_rank,empty_1,country_code,name,empty,id,points,tournaments_played,continent,country,date,draw
0,1,NaN,NaN,DEN,[DEN] Viktor AXELSEN,NaN,25831,107455,15,Europe,Denmark,10/24/2023,MS
1,2,NaN,NaN,INA,[INA] Anthony Sinisuka GINTING,NaN,95661,86611,19,Asia,Indonesia,10/24/2023,MS
2,3,NaN,NaN,JPN,[JPN] Kodai NARAOKA,NaN,62063,82015,23,Asia,Japan,10/24/2023,MS
3,4,1.0,NaN,THA,[THA] Kunlavut VITIDSARN,NaN,64032,79778,18,Asia,Thailand,10/24/2023,MS
4,5,1.0,NaN,CHN,[CHN] LI Shi Feng,NaN,95476,78548,23,Asia,China,10/24/2023,MS


,rank,delta_rank,empty_1,country_code,name,empty,id,points,tournaments_played,continent,country,date,draw
29095,96,12,,TPETPE,[TPE] LIU Kuang Heng[TPE] JHENG Yu Chieh,,5960982732,16770,7,Asia,Chinese Taipei,11/26/2024,XD
29096,97,4,,MEXMEX,[MEX] Luis Armando MONTOYA NAVARRO[MEX] Miriam...,,7498795072,16570,6,Pan America,Mexico,11/26/2024,XD
29097,98,2,,INAINA,[INA] Marwan FAZA[INA] Aisyah Salsabila Putri ...,,7278279077,16060,6,Asia,Indonesia,11/26/2024,XD
29098,99,2,,THATHA,[THA] Neuaduang MANGKORNLOI[THA] Atitaya POVANON,,8780859892,15980,9,Asia,Thailand,11/26/2024,XD
29099,100,1,,IRLIRL,[IRL] Joshua MAGEE[IRL] Moya RYAN,,5906858337,15640,15,Europe,Ireland,11/26/2024,XD


### CLEANING DATA

In [210]:
def clean_data(df_input):

    df = df_input.copy()
    
    df['country_code'] = df['country_code'].apply(lambda x: x[0:3] if (len(x) > 4 and x[0:3] == x[3:6]) else x)

    df['date'] = df['date'].apply(lambda x: x[6:10] + '-' + x[0:2] + '-' + x[3:5])

    df['name'] = df['name'].str[5:].str.upper()

    df['name'] = df['name'].apply(lambda x: x.split('[')[0] + ' / ' + x.split(']')[1] if '[' in x else x)

    df['delta_rank'] = df['delta_rank'].apply(lambda x: x if x != '' else None)
    df['delta_rank'] = df['delta_rank'].fillna(0).astype('int')

    df['id'] = df['id'].apply(lambda x: str(x)[0:5]+' / '+str(x)[5:10] if len(str(x)) > 5 else str(x))

    df = df.drop(['empty', 'empty_1'], axis = 1)

    return df

In [212]:
df_delta_clean = clean_data(df_delta)
df_delta_clean

,rank,delta_rank,country_code,name,id,points,tournaments_played,continent,country,date,draw
0,1,0,DEN,VIKTOR AXELSEN,25831,107455,15,Europe,Denmark,2023-10-24,MS
1,2,0,INA,ANTHONY SINISUKA GINTING,95661,86611,19,Asia,Indonesia,2023-10-24,MS
2,3,0,JPN,KODAI NARAOKA,62063,82015,23,Asia,Japan,2023-10-24,MS
3,4,1,THA,KUNLAVUT VITIDSARN,64032,79778,18,Asia,Thailand,2023-10-24,MS
4,5,1,CHN,LI SHI FENG,95476,78548,23,Asia,China,2023-10-24,MS
...,...,...,...,...,...,...,...,...,...,...,...
29095,96,12,TPE,LIU KUANG HENG / JHENG YU CHIEH,59609 / 82732,16770,7,Asia,Chinese Taipei,2024-11-26,XD
29096,97,4,MEX,LUIS ARMANDO MONTOYA NAVARRO / MIRIAM JACQUE...,74987 / 95072,16570,6,Pan America,Mexico,2024-11-26,XD
29097,98,2,INA,MARWAN FAZA / AISYAH SALSABILA PUTRI PRANATA,72782 / 79077,16060,6,Asia,Indonesia,2024-11-26,XD
29098,99,2,THA,NEUADUANG MANGKORNLOI / ATITAYA POVANON,87808 / 59892,15980,9,Asia,Thailand,2024-11-26,XD


# MERGING DATA

In [214]:
df_delta_clean['date'].min(), df_old['date'].max()

('2023-10-24', '2023-10-16')

In [216]:
df_final = pd.concat([df_delta_clean, df_old], ignore_index=True)

In [221]:
df_final = df_final.drop_duplicates()

In [224]:
df_final.to_csv('bwf_historic_rankings.csv')

In [225]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 888500 entries, 0 to 888599
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   rank                888500 non-null  object 
 1   delta_rank          29000 non-null   float64
 2   country_code        29000 non-null   object 
 3   name                888497 non-null  object 
 4   id                  29000 non-null   object 
 5   points              888500 non-null  object 
 6   tournaments_played  29000 non-null   object 
 7   continent           29000 non-null   object 
 8   country             888470 non-null  object 
 9   date                888500 non-null  object 
 10  draw                888500 non-null  object 
dtypes: float64(1), object(10)
memory usage: 81.3+ MB


## SCRAPPING TOURNAMENTS DATA

In [70]:
driver = webdriver.Chrome()
l_final = []
months = ['JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER']

for year in range(2018, 2024):
    
    driver.get('https://bwfworldtour.bwfbadminton.com/calendar/?cyear='+str(year))
    
    l_tour = driver.find_element(By.XPATH, '//*[@id="page"]/div/div[1]/div/div/div/div').text.split('\n')
    l_tour = [x for x in l_tour if x not in months]
    
    for x in range(5, len(l_tour), 5):
    
        l_final.append(l_tour[x-5:x])

In [71]:
df = pd.DataFrame(l_final)
df

,0,1,2,3,4
0,PRINCESS SIRIVANNAVARI Thailand Masters 2018,09 - 14 JANUARY,"PRIZE MONEY USD $150,000",Bangkok,Thailand
1,PERODUA Malaysia Masters 2018,16 - 21 JANUARY,"PRIZE MONEY USD $350,000",Bukit Jalil,Malaysia
2,DAIHATSU Indonesia Masters 2018,23 - 28 JANUARY,"PRIZE MONEY USD $350,000",Jakarta,Indonesia
3,YONEX-SUNRISE DR. AKHILESH DAS GUPTA India Ope...,30 JANUARY - 04 FEBRUARY,"PRIZE MONEY USD $350,000",New Delhi,India
4,YONEX Swiss Open 2018,20 - 25 FEBRUARY,"PRIZE MONEY USD $150,000",Basel,Switzerland
...,...,...,...,...,...
106,SimInvest Indonesia Open 2021(New dates),23 - 28 NOVEMBER,"PRIZE MONEY USD $850,000",Bali,Indonesia
107,Korea Masters 2023,07 - 12 NOVEMBER,"PRIZE MONEY USD $210,000",Gwangju,Korea
108,Kumamoto Masters Japan 2023,14 - 19 NOVEMBER,"PRIZE MONEY USD $420,000",Kumamoto,Japan
109,China Masters 2023,21 - 26 NOVEMBER,"PRIZE MONEY USD $1,000,000",Shenzhen,China


In [41]:
lnks=driver.find_elements(By.TAG_NAME, "a")

for lnk in lnks:
   if "tournament" in lnk.get_attribute('href'):
      print(lnk.get_attribute('href'))

https://bwfworldtour.bwfbadminton.com/tournament/3139/princess-sirivannavari-thailand-masters-2018
https://bwfworldtour.bwfbadminton.com/tournament/3143/perodua-malaysia-masters-2018
https://bwfworldtour.bwfbadminton.com/tournament/3140/daihatsu-indonesia-masters-2018
https://bwfworldtour.bwfbadminton.com/tournament/3170/yonex-sunrise-dr-akhilesh-das-gupta-india-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3144/yonex-swiss-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3146/yonex-german-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3141/yonex-all-england-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3147/barfoot-thompson-new-zealand-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3148/crown-group-australian-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3216/2018-yonex-us-open
https://bwfworldtour.bwfbadminton.com/tournament/3142/celcom-axiata-malaysia-open-2018
https://bwfworldtour.bwfbadminton.com/tournament/3